In [2]:
import os
import numpy as np
import pandas as pd
import joblib
import datetime
import gradio as gr
import matplotlib.pyplot as plt
import tensorflow as tf
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Load the LSTM model
LSTM_MODEL_PATH = r'E:\Climate_project\amity\gulam_dataset\bilstm_pm25_model.h5'
HOLT_WINTERS_MODEL_PATH = r"E:\Climate_project\amity\gulam_dataset\holts_winter_model.pkl"


# Load LSTM Model
from tensorflow.keras.losses import MeanSquaredError

# Define custom objects
custom_objects = {"mse": MeanSquaredError()}

# Load LSTM Model with custom objects
lstm_model = tf.keras.models.load_model(LSTM_MODEL_PATH, custom_objects=custom_objects)


# Load Holt-Winters Model
holt_winters_model = joblib.load(HOLT_WINTERS_MODEL_PATH)

# Function to preprocess input for LSTM model
def prepare_lstm_input(date):
    """
    Convert the future date into a numerical format suitable for LSTM model input.
    """
    return np.array([[pd.to_datetime(date).toordinal()]])

# Function to predict AQI using LSTM
def predict_with_lstm(future_date):
    """
    Predict AQI using the LSTM model.
    """
    input_data = prepare_lstm_input(future_date)
    prediction = lstm_model.predict(input_data)
    return max(0, prediction[0][0])  # Ensure AQI is non-negative

# Function to predict AQI using Holt-Winters
def predict_with_holt_winters(future_date):
    """
    Predict AQI using the Holt-Winters model.
    """
    future_steps = (pd.to_datetime(future_date) - pd.to_datetime("today")).days
    if future_steps < 0:
        return "⚠️ Date must be in the future!"
    
    forecast = holt_winters_model.forecast(steps=future_steps)
    return max(0, forecast.iloc[-1])  # Ensure AQI is non-negative

# Main function for Gradio UI
def predict_aqi(model_choice, city_name, future_date):
    """
    Predicts AQI based on the selected model.
    """
    if model_choice == "LSTM":
        predicted_aqi = predict_with_lstm(future_date)
    elif model_choice == "Holt-Winters":
        predicted_aqi = predict_with_holt_winters(future_date)
    else:
        return "⚠️ Invalid model choice!", None

    # Generate Plot
    dates = pd.date_range(start=pd.to_datetime("today"), periods=10, freq='D')
    values = [predict_with_lstm(d) if model_choice == "LSTM" else predict_with_holt_winters(d) for d in dates]

    plt.figure(figsize=(6, 4))
    plt.plot(dates, values, marker='o', linestyle='-', label=f"{model_choice} AQI Prediction", color="blue")
    plt.axvline(pd.to_datetime(future_date), color='red', linestyle='dashed', label="Selected Date")
    plt.xlabel("Date")
    plt.ylabel("AQI Index Value")
    plt.title(f"AQI Forecast for {city_name} ({model_choice})")
    plt.legend()
    plt.grid()

    # Return Prediction & Plot
    return f"📈 Predicted AQI for {city_name} on {future_date}: {round(predicted_aqi, 2)}", plt

# Define Gradio UI
interface = gr.Interface(
    fn=predict_aqi,
    inputs=[
        gr.Radio(["LSTM", "Holt-Winters"], label="Select Model"),
        gr.Textbox(label="Enter City Name", placeholder="E.g., New York"),
        gr.Textbox(label="Enter Future Date (YYYY-MM-DD)", placeholder="E.g., 2025-03-01"),
    ],
    outputs=[
        gr.Textbox(label="Prediction Output"),
        gr.Plot(label="AQI Forecast Graph"),
    ],
    title="🌍 AQI Prediction Dashboard (LSTM & Holt-Winters)",
    description="Choose an AI model (LSTM or Holt-Winters) to predict AQI based on historical data.",
)

# Launch Gradio App
if __name__ == "__main__":
    interface.launch(share=False)  # Runs locally


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 773ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import joblib
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
import gradio as gr

def load_model_and_scaler():
    model = tf.keras.models.load_model(r"E:\Climate_project\amity\gulam_dataset\bilstm_pm25_model.h5")
    scaler = joblib.load("scaler.pkl")
    print("Model and scaler loaded successfully!")
    return model, scaler

def predict_future_pm25(city_name, df, model, scaler, look_back, future_date):
    city_data = df[df['City'].str.lower() == city_name.lower()].sort_values(by='Datetime')
    
    if city_data.empty:
        return f"No data available for {city_name}. Check spelling or dataset."
    
    city_data = city_data.dropna(subset=['PM2.5'])
    
    if len(city_data) <= look_back:
        return f"Not enough historical data for {city_name}. Minimum {look_back+1} days required."
    
    city_data['Scaled_PM2.5'] = scaler.transform(city_data[['PM2.5']])
    last_days = city_data['Scaled_PM2.5'].iloc[-look_back:].values.reshape((1, look_back, 1))
    
    future_date = datetime.strptime(future_date, "%Y-%m-%d")
    last_known_date = city_data['Datetime'].max()
    
    if future_date <= last_known_date:
        return "Future date must be beyond the last recorded date in the dataset."
    
    days_ahead = (future_date - last_known_date).days
    
    for _ in range(days_ahead):
        pred_scaled = model.predict(last_days)[0, 0]
        last_days = np.roll(last_days, -1)
        last_days[0, -1, 0] = pred_scaled
    
    pred_pm25 = scaler.inverse_transform([[pred_scaled]])[0][0]
    
    return round(pred_pm25, 2)

def gradio_predict(city_name, future_date):
    df = pd.read_csv("sorted_aqi_hourly_dataset.csv")
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    look_back = 30  # Ensure consistency with training
    
    model, scaler = load_model_and_scaler()
    pred_pm25 = predict_future_pm25(city_name, df, model, scaler, look_back, future_date)
    
    future_actual = df[(df['City'].str.lower() == city_name.lower()) & (df['Datetime'] == future_date)]
    if not future_actual.empty:
        actual_pm25 = future_actual['PM2.5'].values[0]
        mse = mean_squared_error([actual_pm25], [pred_pm25])
        return f"Predicted PM2.5 for {city_name} on {future_date}: {pred_pm25}\nMSE: {mse:.4f}"
    else:
        return f"Predicted PM2.5 for {city_name} on {future_date}: {pred_pm25}\n(No actual data available for MSE calculation)"

interface = gr.Interface(
    fn=gradio_predict,
    inputs=[
        gr.Textbox(label="Enter City Name", placeholder="E.g., New York"),
        gr.Textbox(label="Enter Future Date (YYYY-MM-DD)", placeholder="E.g., 2025-03-01"),
    ],
    outputs=gr.Textbox(label="Prediction Output"),
    title="🌍 PM2.5 Prediction Dashboard",
    description="Predict PM2.5 levels for a given city using BiLSTM.",
)

if __name__ == "__main__":
    interface.launch(share=False)


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "d:\Anacondaa\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anacondaa\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anacondaa\Lib\site-packages\gradio\blocks.py", line 2108, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anacondaa\Lib\site-packages\gradio\blocks.py", line 1655, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anacondaa\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
import joblib
import pickle
import gradio as gr

# Load and preprocess dataset
def load_data(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'])
    df['hour'] = df['date'].dt.hour
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    return df

# Define file path
file_path = r'C:\Users\ASUS\Desktop\Climate1\delhi_aqi.csv'
df = load_data(file_path)

# Define target variable
features = ['hour', 'day', 'month', 'year']
target = 'pm2_5'

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, shuffle=False)

# Normalize features for LSTM
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for LSTM
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Train XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
xgb_model.fit(X_train, y_train)

# Train LSTM model
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(1, 4)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)
])

lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train_lstm, y_train, epochs=20, batch_size=32, validation_data=(X_test_lstm, y_test), verbose=1)

# Save the models
xgb_model.save_model("xgb_model.json")
lstm_model.save("lstm_model.h5")
joblib.dump(scaler, "scaler.pkl")

# Save ensemble model
ensemble_model_data = {
    "xgb_model": "xgb_model.json",
    "lstm_model": "lstm_model.h5",
    "scaler": "scaler.pkl",
    "weights": (0.5, 0.5)
}
with open("ensemble_model3.pkl", "wb") as f:
    pickle.dump(ensemble_model_data, f)

# Function to load the models
def load_models():
    with open("ensemble_model3.pkl", "rb") as f:
        ensemble_data = pickle.load(f)
    
    xgb_loaded = xgb.XGBRegressor()
    xgb_loaded.load_model(ensemble_data["xgb_model"])
    lstm_loaded = load_model(ensemble_data["lstm_model"])
    scaler_loaded = joblib.load(ensemble_data["scaler"])
    weights = ensemble_data["weights"]
    
    return xgb_loaded, lstm_loaded, scaler_loaded, weights

# Function to predict PM2.5 for a future date
def predict_future_pm25(future_date):
    xgb_model, lstm_model, scaler, weights = load_models()
    
    # Convert future_date string to datetime object
    future_date = pd.to_datetime(future_date)
    
    # Extract time-based features
    future_features = [future_date.hour, future_date.day, future_date.month, future_date.year]
    
    # Preprocess input
    future_scaled = scaler.transform([future_features])
    future_lstm = future_scaled.reshape((1, 1, len(future_features)))
    
    # Make predictions
    pred_xgb = xgb_model.predict(future_scaled)[0]
    pred_lstm = lstm_model.predict(future_lstm)[0][0]
    
    # Ensemble prediction
    pred_ensemble = (weights[0] * pred_xgb) + (weights[1] * pred_lstm)
    
    return f"Predicted PM2.5 for {future_date}: {pred_ensemble:.2f}"

# Gradio interface
def gradio_interface(future_date):
    return predict_future_pm25(future_date)

iface = gr.Interface(fn=gradio_interface, inputs="text", outputs="text", title="Future PM2.5 Prediction")
iface.launch()

d:\Anacondaa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 95398.0547 - val_loss: 58008.0586
Epoch 2/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 51642.0078 - val_loss: 53663.7344
Epoch 3/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 50370.0820 - val_loss: 53842.3359
Epoch 4/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 52237.2930 - val_loss: 53446.0547
Epoch 5/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 53328.0703 - val_loss: 54110.2656
Epoch 6/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 50414.6094 - val_loss: 53270.2734
Epoch 7/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 53085.2812 - val_loss: 53556.9219
Epoch 8/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 49965.8086 - val_loss: 53146.0078
Epoch 9/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 51185.7305 - val_loss: 53361.9414
Epoch 10/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 51553.8789 - val_loss: 53437.8398
Epoch 11/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


d:\Anacondaa\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 560ms/step


d:\Anacondaa\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 497ms/step


d:\Anacondaa\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
